In [ ]:
### Additive Regression 
$$f_c \sim \cal{GP}(0,k_c),\; \forall c \in [1..C]$$
$$y^{(n)}|f_1...f_C,x^{(n)} = y^{(n)}|\sum_c f_c(x^{(n)}_c)$$

Functions $f_c$ are all functions of separate covariates $x_c$

In [ ]:
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import cm
tf.logging.set_verbosity(tf.logging.ERROR)
from tensorflow.contrib.opt import ScipyOptimizerInterface as soi
np.random.seed(10)

In [ ]:
## Simulating synthetic data

In [ ]:
lik = 'Poisson'
#lik = 'Gaussian'
assert lik in ['Poisson','Gaussian']

#---------------------------------------------------
# Declaring additive GP model parameters
N =500
D = 4 # covariates dimension
R = 1 # number of trials
f_indices = [[0],[1,2],[3]]
C = len(f_indices) # number of latent functions
scale = 2.
fs = [lambda x:np.sin(x)**3*scale, 
      lambda x: (np.sin(x[:,0])*np.sin(x[:,1])).reshape(-1,1)*scale ,
     lambda x:np.cos(x)*scale]
#---------------------------------------------------
# Simulating data
xmin,xmax=-3,3
X_np = np.random.uniform(xmin,xmax,(N,D))
F_np = np.hstack([fs[d](X_np[:,f_indices[d]]) for d in range(C)])
pred_np = np.sum(F_np,axis=1,keepdims=True)

if lik == 'Gaussian':
    Y_np = pred_np + np.random.randn(N,R)*.5
elif lik=='Poisson':
    link = np.exp
    rate = np.tile(link(pred_np),[1,R])
    Y_np = np.random.poisson(rate,size=(N,R))



In [ ]:
colors_c = plt.cm.winter(np.linspace(0,1,C))
fig,ax = plt.subplots(1,C,figsize=(C*4,4))
for c in range(C):
    i = f_indices[c]
    if len(f_indices[c])==1:
        o = np.argsort(X_np[:,f_indices[c]],0)
        ax[c].plot(X_np[o,i],F_np[o,c],'-',color=colors_c[c])
        ax[c].set_xlabel('$x_%d$'%i[0],fontsize=20)
        ax[c].set_ylabel('$f_%d(x_%d)$'%(i[0],i[0]),fontsize=20)
    elif len(f_indices[c])==2:
        ax[c].scatter(X_np[:,i[0]],
                      X_np[:,i[1]],
                      c=F_np[:,c],linewidth=0)
        ax[c].set_xlabel('$x_%d$'%i[0],fontsize=20)
        ax[c].set_ylabel('$x_%d$'%i[1],fontsize=20)
        ax[c].set_title('$f(x_%d,x_%d)$'%(i[0],i[1]),fontsize=20)
plt.suptitle('True underlying functions',y=1.05,fontsize=20)   
fig.tight_layout()
plt.show()



In [ ]:
## Constructing tensorflow model

In [ ]:
import sys
sys.path.append('../SVAGP')
from kernels import RBF
from likelihoods import Gaussian, Poisson, Gaussian_with_link
from settings import np_float_type,int_type
from model import SVAGP

#---------------------------------------------------
# Constructing tensorflow model

X = tf.placeholder(tf.float32,[N,D])
Y = tf.placeholder(tf.float32,[N,R])
ks,Zs = [],[]

ks =[]
with tf.variable_scope("kernels") as scope:
    for c in range(C):
        with tf.variable_scope("kernel%d"%c) as scope:
            input_dim = len(f_indices[c])
            ks.append(  RBF(input_dim,lengthscales=.5*np.ones(input_dim),  variance=1.))    
    
with tf.variable_scope("likelihood") as scope:
    if lik=='Gaussian':
        likelihood = Gaussian(variance=1)
    elif lik == 'Poisson':
        likelihood = Poisson()
    
with tf.variable_scope("ind_points") as scope:
    for c in range(C):
        with tf.variable_scope("ind_points%d"%c) as scope:
            input_dim = len(f_indices[c])
            Z_ = np.random.uniform(xmin,xmax,[20,input_dim]).astype(np_float_type)
            Zs.append(   tf.Variable(Z_,tf.float32,name='Z') )


with tf.variable_scope("model") as scope:
    m= SVAGP(X,Y,ks,likelihood,Zs,q_diag=True,f_indices=f_indices)

    
    

In [ ]:
## Running inference and learning

In [ ]:

#---------------------------------------------------

sess  = tf.Session()
sess.run(tf.global_variables_initializer()) # reset values to wrong
# declare loss
loss = -m.build_likelihood()
# separate variables
vars_e, vars_m, vars_h, vars_z= [], [], [], []
vars_e += tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='model/inference')
if lik=='Gaussian':
    vars_m += tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='likelihood')
vars_z += tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='ind_points')
vars_h += tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='kernels')
# declare optimizers
opt_e = soi(loss, var_list=vars_e,  method='L-BFGS-B', options={'ftol': 1e-4})
opt_m = soi(loss, var_list=vars_m,  method='L-BFGS-B', options={'ftol': 1e-4})
opt_z = soi(loss, var_list=vars_z,  method='L-BFGS-B', options={'ftol': 1e-4})
opt_h = soi(loss, var_list=vars_h,  method='L-BFGS-B', options={'ftol': 1e-2})

init = tf.global_variables_initializer()
sess.run(init) # reset values to wrong
feed_dic = {Y:Y_np, X:X_np}

#---------------------------------------------------


print('Optimized variables:')
for var in vars_e+vars_z+vars_h:
    print(var.name)  # Prints the name of the variable alongside its val

nit = 30
loss_array = np.zeros((nit,))


# declare which optimization to perform

OPT = ['E','Z','H']
if lik=='Gaussian':
    OPT.append('M')
    
# Optimization is performed using L-BFGS-B, iterating over different subsets of variable
# - E: inference (as in classical EM)
# - Z: update of inducing point locations
# - H: kernel hyperparameter optimization

print('Starting Optimization')
opt_e.minimize(sess, feed_dict=feed_dic)
if 'E' in OPT:
    opt_e.minimize(sess, feed_dict=feed_dic)
if 'H' in OPT:
    opt_h.minimize(sess, feed_dict=feed_dic)

for it in range( nit):
    if 'E' in OPT:    
        opt_e.minimize(sess, feed_dict=feed_dic)
    if 'M' in OPT:    
        opt_m.minimize(sess, feed_dict=feed_dic)
    if 'Z' in OPT:
        opt_z.minimize(sess, feed_dict=feed_dic)
    if 'H' in OPT:
        opt_z.minimize(sess, feed_dict=feed_dic)

    loss_array[it]= float(sess.run(loss, feed_dic))

Fs_mean,Fs_var = sess.run(m.build_predict_fs(X),  feed_dic)
pred_mean,pred_var = sess.run(m.build_predict_additive_predictor(X),  feed_dic)
Zs = sess.run(m.Zs,  feed_dic)

sess.close()



In [ ]:
fig,axarr = plt.subplots(1,2,figsize=(8,4))
ax=axarr[0]
ax.plot(loss_array[:it], linewidth=3, color='blue')
ax.set_xlabel('iterations',fontsize=20)
ax.set_ylabel('Variational Objective',fontsize=20)
ax=axarr[1]
ax.errorbar(pred_mean,pred_np,yerr=np.sqrt(pred_var), 
            elinewidth = 1, fmt='.', color='blue', alpha=.1)
ax.plot(pred_mean,pred_np,'.',color='blue')
ax.plot([pred_mean.min(),pred_mean.max()],
        [pred_mean.min(),pred_mean.max()],
        '--',linewidth=2,color='grey')
ax.set_xlabel('True predictor',fontsize=20)
ax.set_ylabel('Predicted predictor',fontsize=20)
fig.tight_layout()
plt.show()
plt.close()


fig,ax = plt.subplots(1,C,figsize=(C*5,5))
for c in range(C):
    i = f_indices[c]
    if len(i)==1:
        o = np.argsort(X_np[:,i],0)
        f,s =  Fs_mean[c,:,0],np.sqrt(Fs_var[c,:,0])
        ax[c].vlines(Zs[c],ymin=f.min(),ymax=f.max(),alpha=.05,color=colors_c[c])
        ax[c].plot(X_np[o,i],f[o],color=colors_c[c])
        ax[c].fill_between(X_np[o,i].flatten(),
                           (f-s)[o].flatten(),
                           y2=(f+s)[o].flatten(),
                           alpha=.1,facecolor=colors_c[c])
        ax[c].plot(X_np[o,i],F_np[o,c],'--',color=colors_c[c])
        ax[c].set_xlabel('$x_%d$'%i[0],fontsize=20)
        ax[c].set_ylabel('$f_%d(x_%d)$'%(i[0],i[0]),fontsize=20)

    elif len(f_indices[c])==2:
        ax[c].scatter(X_np[:,i[0]],
                      X_np[:,i[1]],
                      c=Fs_mean[c,:,0],linewidth=0)
        ax[c].scatter(Zs[c][:,0],Zs[c][:,1],
                      c='r', marker=(5, 1))
        ax[c].set_xlabel('$x_%d$'%i[0],fontsize=20)
        ax[c].set_ylabel('$x_%d$'%i[1],fontsize=20)
        ax[c].set_title('$f(x_%d,x_%d)$'%(i[0],i[1]),fontsize=20)
plt.suptitle('Inferred underlying functions',y=1.05,fontsize=20) 
fig.tight_layout()

plt.show()